In [6]:
import xarray as xr
import glob
import pandas as pd
import numpy as np
import torch
import torch.nn as nn

In [3]:
# little helper function
def xr_opener(file):
    ds = xr.open_dataset(file)
    if 'index' in ds.coords:
        ds = ds.rename({'index': 'date'}) 
    return ds

#### Read all files

In [4]:
geo_folder = '../geo_data/great_db'

In [7]:
files = glob.glob(f'{geo_folder}/nc_concat/*.nc')
big_file = xr.concat([xr_opener(file) for file in files],
                     dim='gauge_id')
big_file


<xarray.Dataset>
Dimensions:     (date: 4749, gauge_id: 1817)
Coordinates:
  * date        (date) datetime64[ns] 2008-01-01 2008-01-02 ... 2020-12-31
  * gauge_id    (gauge_id) object '5746' '75239' '77339' ... '3001' '70207'
Data variables: (12/23)
    lvl_sm      (gauge_id, date) float64 176.0 176.0 176.0 ... 92.0 93.0 92.0
    q_cms_s     (gauge_id, date) float64 4.78 4.66 4.54 4.42 ... nan nan nan nan
    lvl_mbs     (gauge_id, date) float64 45.43 45.43 45.43 ... 119.6 119.6 119.6
    q_mm_day    (gauge_id, date) float64 nan nan nan nan nan ... nan nan nan nan
    t_max_e5l   (gauge_id, date) float64 -8.21 -12.95 -12.35 ... -19.85 -17.85
    t_max_e5    (gauge_id, date) float64 -6.927 -10.66 -12.57 ... -18.71 -17.64
    ...          ...
    SMroot      (gauge_id, date) float64 0.3179 0.3177 0.3176 ... 0.3525 0.3525
    Ew          (gauge_id, date) float64 0.0002926 0.0004143 ... 0.0001287 0.0
    Ei          (gauge_id, date) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    S           (gauge_id, date) float64 0.2933 0.2981 0.8762 ... 1.0 1.0 1.0
    E           (gauge_id, date) float64 0.002709 0.01003 ... 0.0001237 -0.02626
    Ep          (gauge_id, date) float64 0.00282 0.01029 ... 0.0001237 -0.02626

##### Select by gauge

In [8]:
gauges = ['5746', '3001', '1001']
ds = big_file.sel(gauge_id=gauges)
# sel is possible on date coordinate as well
ds

<xarray.Dataset>
Dimensions:     (date: 4749, gauge_id: 3)
Coordinates:
  * date        (date) datetime64[ns] 2008-01-01 2008-01-02 ... 2020-12-31
  * gauge_id    (gauge_id) object '5746' '3001' '1001'
Data variables: (12/23)
    lvl_sm      (gauge_id, date) float64 176.0 176.0 176.0 ... 228.0 228.0 228.0
    q_cms_s     (gauge_id, date) float64 4.78 4.66 4.54 4.42 ... nan nan nan nan
    lvl_mbs     (gauge_id, date) float64 45.43 45.43 45.43 ... 480.5 480.5 480.5
    q_mm_day    (gauge_id, date) float64 nan nan nan nan nan ... nan nan nan nan
    t_max_e5l   (gauge_id, date) float64 -8.21 -12.95 -12.35 ... -28.55 -32.34
    t_max_e5    (gauge_id, date) float64 -6.927 -10.66 -12.57 ... -26.1 -29.87
    ...          ...
    SMroot      (gauge_id, date) float64 0.3179 0.3177 0.3176 ... 0.3972 0.3972
    Ew          (gauge_id, date) float64 0.0002926 0.0004143 ... 7.016e-07 0.0
    Ei          (gauge_id, date) float64 0.0 0.0 0.0 ... 9.843e-06 0.0005234
    S           (gauge_id, date) float64 0.2933 0.2981 0.8762 ... 0.9976 0.9976
    E           (gauge_id, date) float64 0.002709 0.01003 ... -0.01616 -0.0184
    Ep          (gauge_id, date) float64 0.00282 0.01029 ... -0.01616 -0.0184

#### Select by other variables and split data

In [9]:
predictors = ['t_max_e5l', 't_max_e5', 't_min_e5l', 't_min_e5', 'prcp_e5l',
             'prcp_e5', 'prcp_gpcp', 'prcp_imerg', 'prcp_mswep', 'Eb', 'Es',
             'Et', 'SMsurf', 'SMroot', 'Ew', 'Ei', 'S', 'E', 'Ep']
target = ['lvl_sm', 'q_cms_s', 'lvl_mbs', 'q_mm_day']

future_window = 7
past_window = 365

train_start = '01/01/2008'
train_end = '12/31/2015'

val_start = '01/01/2016'
val_end = '12/31/2018'

test_start = '01/01/2019'
test_end = '12/31/2020'

In [10]:
ds.sel(date=slice(train_start, train_end))

<xarray.Dataset>
Dimensions:     (date: 2922, gauge_id: 3)
Coordinates:
  * date        (date) datetime64[ns] 2008-01-01 2008-01-02 ... 2015-12-31
  * gauge_id    (gauge_id) object '5746' '3001' '1001'
Data variables: (12/23)
    lvl_sm      (gauge_id, date) float64 176.0 176.0 176.0 ... 67.0 68.0 69.0
    q_cms_s     (gauge_id, date) float64 4.78 4.66 4.54 4.42 ... nan nan nan nan
    lvl_mbs     (gauge_id, date) float64 45.43 45.43 45.43 ... 478.9 478.9 478.9
    q_mm_day    (gauge_id, date) float64 nan nan nan nan nan ... nan nan nan nan
    t_max_e5l   (gauge_id, date) float64 -8.21 -12.95 -12.35 ... -28.11 -33.03
    t_max_e5    (gauge_id, date) float64 -6.927 -10.66 -12.57 ... -29.52 -34.88
    ...          ...
    SMroot      (gauge_id, date) float64 0.3179 0.3177 0.3176 ... 0.4207 0.4207
    Ew          (gauge_id, date) float64 0.0002926 0.0004143 ... 0.0 0.0
    Ei          (gauge_id, date) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    S           (gauge_id, date) float64 0.2933 0.2981 0.8762 ... 0.9976 0.9976
    E           (gauge_id, date) float64 0.002709 0.01003 ... -0.006186
    Ep          (gauge_id, date) float64 0.00282 0.01029 ... -0.009893 -0.006186

In [13]:
ds.sel(date=slice(train_start, train_end),
       gauge_id='5746')[target]

<xarray.Dataset>
Dimensions:   (date: 2922)
Coordinates:
  * date      (date) datetime64[ns] 2008-01-01 2008-01-02 ... 2015-12-31
    gauge_id  <U4 '5746'
Data variables:
    lvl_sm    (date) float64 176.0 176.0 176.0 176.0 ... 171.0 171.0 171.0 171.0
    q_cms_s   (date) float64 4.78 4.66 4.54 4.42 4.3 ... 3.55 3.35 3.15 2.95
    lvl_mbs   (date) float64 45.43 45.43 45.43 45.43 ... 45.38 45.38 45.38 45.38
    q_mm_day  (date) float64 nan nan nan nan nan nan ... nan nan nan nan nan nan

In [ ]:
def split_ds(ds: xr.Dataset) -> dict:
    past_predictors = list()
    future_targets = list()

    for i in range(past_window, len(ds['date'])-future_window):
        
        t = ds['date'].isel(date=i).values
        t = pd.to_datetime(t).strftime('%Y-%m-%d')
        
        past_predictors.append(
            (ds.to_array().values[:, :, i-past_window:i]))
        
        future_targets.append(
            (ds[target].to_array().values[:, :, i:i+future_window]))

    past_predictors = np.array(past_predictors)
    future_targets = np.array(future_targets)
    
    data = {'past_seq': torch.tensor(past_predictors, dtype=torch.float),
            'future_seq': torch.tensor(future_targets, dtype=torch.float)}
    
    return data


In [ ]:
# train
train_past, train_future = split_ds(train_ds)
# validation
val_past, val_future = split_ds(val_ds)
# test
test_past, test_future = split_ds(test_ds)